In [2]:
!pip3 install -r requirements.txt
!pip3 install yfinance --upgrade --no-cache-dir

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.4/499.4 kB 6.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 6.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 7.9 MB/s eta 0:00:00
  Attempting uninstall: pytz
    Found existing installation: pytz 2021.3
    Uninstalling pytz-2021.3:
      Successfully uninstalled pytz-2021.3
  Attempting uninstall: lxml
    Found existing installation: lxml 4.6.3
    Uninstalling lxml-4.6.3:
      Successfully uninstalled lxml-4.6.3
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.10.0
    Uninstalling beautifulsoup4-4.10.0:
      Successfully uninstalled beautifulsoup4-4.10.0
  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.1.84
    U

In [3]:
import pandas as pd
import numpy as np
import yfinance as yf

In [5]:
# session = requests_cache.CachedSession(cache_name='cache', backend='sqlite')

# just add headers to your session and provide it to the reader
# session.headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0', 'Accept': 'application/json;charset=utf-8'}

def get_data_for_multiple_stocks(tickers,start_date,end_date):
    
    stocks = dict()
    # loop through all the tickers
    for ticker in tickers:
        
        # get the data for the specific ticker
        # s = DataReader(ticker, 'stooq', start_date, end_date, session=session)
        s = yf.download(ticker, start_date, end_date)
        #stocks.append(s.rename(mapper={'Adj Close': ticker}, axis=1)[ticker])
        s.insert(0, "Ticker", ticker)  #insert ticker column so you can reference better later
        
        s['Prev Close'] = s['Adj Close'].shift(1)
        s['log_return'] = np.log(s['Adj Close']/s['Prev Close'])
        s['perc_return'] = (s['Adj Close']/s['Prev Close']) - 1
        
        # add it to the dictionary
        stocks[ticker] = s
    # return the dictionary
    #return pd.concat(stocks, axis=1)
    return stocks

## Proxies for Energy Prices
### 1. WTI Crude Oil Futures (US produced)
- WTI crude oil is a light, sweet crude oil that is primarily produced in the United States. The WTI index reflects the price of a barrel of WTI crude oil traded on the New York Mercantile Exchange (NYMEX) and is often used as a benchmark for pricing other types of crude oil, as well as for gasoline, diesel fuel, and other refined products.

### 2. Brent Crude Oil Futures (EMEA produced)
- The Brent crude oil index, also known as the Brent crude oil benchmark, is one of the most widely referenced global oil benchmarks. The index reflects the price of a barrel of crude oil produced from the North Sea (primarily the Brent oilfield), and is traded on the Intercontinental Exchange (ICE).


In [6]:
# Import dataset 

tickers = ['CL=F', 'BZ=F']
portfolio =  get_data_for_multiple_stocks(tickers, start_date='2019-01-01',end_date= '2022-12-31')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [7]:
cl_f = portfolio['CL=F'].reset_index()

# Make Date to date type
cl_f['Date'] = cl_f['Date'].apply(str)
cl_f['Date'] = pd.to_datetime(cl_f['Date'], infer_datetime_format=True)

bz_f = portfolio['BZ=F'].reset_index()

# Make Date to date type
bz_f['Date'] = bz_f['Date'].apply(str)
bz_f['Date'] = pd.to_datetime(cl_f['Date'], infer_datetime_format=True)

In [9]:
bz_f.to_csv("extracted/brentCrudeOil_2019_2022.csv")
cl_f.to_csv("extracted/WTI_2019_2022.csv")